In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import glob

Preprocess hmmer out with tab script

In [ ]:
#from a blast search using inteins as a query and stanard blast output
hwa = pd.read_table('hwa.htab', sep='\t', lineterminator='\n', skiprows=0)
hwa

In [ ]:
#get all n-terms, need to script _ instead of space and also get rid of #
for indy,row in hwa.iterrows():
    if row['hmm _oord_from'] > 25:
        hwa.drop(index=indy, inplace=True)
hwa

In [ ]:
group = hwa.groupby('target_name')
df2 = group.apply(lambda x: x['ali_coord_from'].unique())
df2


In [ ]:
HEN = pd.read_csv("hwa_protein.blocked", sep='\t',lineterminator='\n', header=None)
HEN.columns = ['gene', 'motif', 'coord']
hen_groups = HEN.groupby('gene')
df3 = hen_groups.apply(lambda x: x['coord'].unique())
intein_doms = {}
for gene,c_list in df3.iteritems():
    c_list.sort()
    c_index =0 
    c_iter = 1
    while c_index < len(c_list):
        if c_iter < len(c_list):
            if c_list[c_iter] < (c_list[c_index] + 471):
                c_list = np.delete(c_list, c_iter)
                
        c_index += 1
        c_iter += 1
    intein_doms[gene] = (c_list)
intein_doms

In [ ]:
HEN = pd.read_csv("hwa_protein.blocked", sep='\t',lineterminator='\n', header=None)
HEN.columns = ['gene', 'motif', 'coord']
hen_groups = HEN.groupby('gene')
df3 = hen_groups.apply(lambda x: x['coord'].unique())
intein_doms = {}
for gene,c_list in df3.iteritems():
    c_list.sort()
    c_index =0 
    c_iter = 1
    empty = []
    while c_index < len(c_list):
        if c_iter < len(c_list):
            if c_list[c_iter] < (c_list[c_index] + 471):
                c_list = np.delete(c_list, c_iter)
                c_index -= 1
                c_iter -= 1
        c_index += 1
        c_iter += 1
    intein_doms[gene] = (c_list)
intein_doms

In [ ]:
for f in intein_doms:
    print(len(intein_doms[f]))

In [ ]:
multi_inteins = 0
multi_hens = 0
for f,j in df2.iteritems():
    if len(j) > 1:
        multi_inteins += 1
        if f in HEN.values:
            multi_hens += 1
        
        

In [ ]:
multi_hens

In [ ]:
multi_inteins 
import os

In [ ]:
for fas in glob.glob("*.htab"):
    genome_name = fas[:13]
    df2 = genome_name + "2"
    
    genome_name = pd.read_table(fas, sep='\t', lineterminator='\n', skiprows=0)
    for indy,row in genome_name.iterrows():
        if row['hmm _oord_from'] > 25:
            genome_name.drop(index=indy, inplace=True)
    
    group = genome_name.groupby('target_name')
    df2 = group.apply(lambda x: x['ali_coord_from'].unique())
    #print(fas[:-5] + "_protein.blocked")
    if os.stat(fas[:-5] + "_protein.blocked").st_size != 0:
        HEN = pd.read_csv(fas[:-5] + "_protein.blocked", sep='\t',lineterminator='\n', header=None)
        HEN.columns = ['gene', 'motif', 'coord']
        hen_groups = HEN.groupby('gene')
        df3 = hen_groups.apply(lambda x: x['coord'].unique())
        intein_doms = {}
        for gene,c_list in df3.iteritems():
            c_list.sort()
            c_index =0 
            c_iter = 1
            while c_index < len(c_list):
                if c_iter < len(c_list):
                    if c_list[c_iter] < (c_list[c_index] + 471):
                        c_list = np.delete(c_list, c_iter)
                        c_index -= 1
                        c_iter -= 1
                c_index += 1
                c_iter += 1
            intein_doms[gene] = (c_list)

    multi_hens = 0
    multi_inteins = 0
    single = 0
    single_hen =0
    multiple = 0
    multiple_hen = 0
    for f,j in df2.iteritems():
        
        if len(j) > 1:
            multiple = multiple + len(j)
            multi_inteins += 1
            if f in HEN.values:
                multi_hens += 1
            if f in intein_doms.keys():
                
                multiple_hen = multiple_hen + len(intein_doms[f])
        if len(j) == 1:
            single += 1
            if f in HEN.values:
                single_hen += 1
    print(fas+ "\t" + str(multi_inteins) + "\t" + str(multi_hens) + "\t" + str(single) + "\t" + str(single_hen) + "\t" + str(multiple) + "\t" + str(multiple_hen), file=open("newcutoffs.tab",'a'))

In [ ]:
genomekey = pd.read_csv("ftpdirpaths.txt", sep='\t',lineterminator='\n', header=None)
genomekey.columns = ['name', 'accession']
replacer = {}
for index,row in genomekey.iterrows():
    replacer[row['accession']] = row['name']
replacer

In [ ]:
archtab2 = pd.read_csv("newcutoffs.tab", sep='\t',lineterminator='\n', header=None)
archtab2.columns = ['name', 1, 2,3, 4, 5, 6]

In [ ]:
for index,row in archtab2.iterrows():
    archtab2.set_value(index, 'name', replacer[row['name']])
archtab2.to_csv('named_newcutoff.csv', sep='\t', index_label=None, line_terminator='\n')

In [ ]:
cd ../full_run1/